In [9]:
################ PLOTTING PRICE TARGETS/INVESTMENT SCENARIOS AND TAX CALCULATIONS WITH MATPLOTLIB/PANDAS ###################

import locale
import pandas as pd
from IPython.display import display

locale.setlocale(locale.LC_ALL, '')

class Marital:
    SINGLE = "Single"
    MARRIED = "Married"
    
class HoldingPeriod:
    LONGTERM = "Long Term" # year or longer
    SHORTTERM = "Short Term" # year or less


#### MAIN PARAMETERS FOR OUTPUT ####

# In Dollars
LONG_TERM_SHARES_OWNED = 100
SHORT_TERM_SHARES_OWNED = 10 # Change if purchasing recently (shares held less than a year)
TOTAL_SHARES_OWNED = LONG_TERM_SHARES_OWNED + SHORT_TERM_SHARES_OWNED
TOTAL_INITIAL_INVESTMENT = 1000
CURRENT_COST_BASIS_PER_SHARE = TOTAL_INITIAL_INVESTMENT / TOTAL_SHARES_OWNED
CURRENT_SHARE_PRICE = 8.00 # change this based on current price to see what you need to invest to get what market valu

FINAL_MARKET_VAL = 2000 # Desired market value for later plotting/capital gains

MARITAL_STATUS_SLIDER_VAL = Marital.SINGLE
ANNUAL_INCOME = 40000
CAPITAL_GAINS = FINAL_MARKET_VAL - TOTAL_INITIAL_INVESTMENT
STATE_TAX_RATE = 0.044 # Enter value for state (decimal)
HOLDING_PERIOD_SLIDER_VAL = HoldingPeriod.LONGTERM


#### END MAIN PARAMETERS FOR OUTPUT ####

## TABLE OUTPUT PARAMETERS ##

DESIRED_MARKET_VALS = [TOTAL_INITIAL_INVESTMENT * 2, TOTAL_INITIAL_INVESTMENT + (TOTAL_INITIAL_INVESTMENT * .10)]
table_titles = ["Investment Doubles", "Increases by 10%"] # should align with above desired market vals to print for their tables

likely_future_price = CURRENT_COST_BASIS_PER_SHARE + .10 # an expected future share price
TARGET_PRICES = [CURRENT_COST_BASIS_PER_SHARE*2, likely_future_price]

## END TABLE OUTPUT PARAMETERS ##

# Style dataframe table output
def printdf(df, title = ""):
    styles = [
        dict(selector="caption",
            props=[
                ("text-align", "center"),
                ("font-size", "125%"),
                ("margin-bottom", ".5rem"),
            ]
        )
    ]
    
    display(
        df
            .style.set_caption(title)
            .set_table_styles(styles)
            .hide(axis="index")
    )

def format_currency(amount):
    return locale.currency(amount, grouping=True)

def calculate_required_stock_price(num_shares, desired_market_value):
    required_price = desired_market_value / num_shares
    return round(required_price, 2)

def compare_for_desired_value(end_value, scenarios, initial_shares):
    data = []

    for scenario in scenarios:
        shares = scenario["shares"]
        purchase_price = scenario['current_price']

        required_price = calculate_required_stock_price(shares, end_value)
        addl_investment = (shares - initial_shares) * purchase_price
        
        data.append({ 
            "Current Share Price": format_currency(purchase_price),
            "Addl Investment": format_currency(addl_investment),
            "Shares Added": shares - initial_shares,
            "Shares Total": shares,
            "Target Sell Price": format_currency(required_price),
        })
    return pd.DataFrame(data)

def required_investment_amt_for(
    desired_market_val,
    target_price,
    purchase_price=CURRENT_SHARE_PRICE,
    current_shares_owned=TOTAL_SHARES_OWNED
):
    '''Returns required addl investment amount to reach the desired market value when the share price is the target price'''
    total_shares_needed = desired_market_val / target_price
    need_to_add_this_many_shares = total_shares_needed - current_shares_owned

    return purchase_price * need_to_add_this_many_shares


def calc_required_investment_amt_for_target_price(desired_market_val, current_price, target_prices):
    data = []

    for target in target_prices:
        addl_investment_needed = required_investment_amt_for(desired_market_val, target_price=target, purchase_price=current_price)

        data.append({
            "Current Share Price": format_currency(current_price),
            "Addl Investment": format_currency(addl_investment_needed),
            "Target Sell Price": format_currency(target),
        })
    return pd.DataFrame(data)


def add_investment(scenario, money_to_add):
    return {"shares": scenario["shares"] + int(money_to_add / scenario["current_price"]), "current_price": scenario["current_price"]}

def create_scenario(current_price, shares_owned=TOTAL_SHARES_OWNED):
    return { "shares": shares_owned, "current_price": current_price }

# Calculate different scenarios where you add a certain amount to the investment to get to a market value goal. 
# The Pandas DataFrame tables show what share price you should sell at to reach the goal given the addl investment.
scenarios = [
    create_scenario(CURRENT_COST_BASIS_PER_SHARE, shares_owned=TOTAL_SHARES_OWNED),
    add_investment(create_scenario(current_price=CURRENT_SHARE_PRICE), money_to_add=100),
    add_investment(create_scenario(current_price=CURRENT_SHARE_PRICE), money_to_add=200),
    add_investment(create_scenario(current_price=CURRENT_SHARE_PRICE), money_to_add=500),
]

def align_dataframes(top_df, bottom_df):
    # Get all unique column names from both DataFrames
    all_columns = pd.Series(top_df.columns.tolist() + bottom_df.columns.tolist()).unique()
    
    # Reindex both DataFrames
    aligned_top_df = top_df.reindex(columns=all_columns, fill_value='')
    aligned_bottom_df = bottom_df.reindex(columns=all_columns, fill_value='')
    
    return aligned_top_df, aligned_bottom_df


#### OUTPUT ###

for i, market_val in enumerate(DESIRED_MARKET_VALS):
    df = compare_for_desired_value(market_val, scenarios, TOTAL_SHARES_OWNED)    
    df2 = calc_required_investment_amt_for_target_price(desired_market_val=market_val, target_prices=TARGET_PRICES, current_price=CURRENT_SHARE_PRICE)

    aligned_top, aligned_bottom = align_dataframes(df, df2)
    printdf(aligned_top, title=f'To Get To: {format_currency(market_val)} ({table_titles[i]})')
    printdf(aligned_bottom, title='Required Addl Investment for Double or Likely Price')

Current Share Price,Addl Investment,Shares Added,Shares Total,Target Sell Price
$9.09,$0.00,0,110,$18.18
$8.00,$96.00,12,122,$16.39
$8.00,$200.00,25,135,$14.81
$8.00,$496.00,62,172,$11.63


Current Share Price,Addl Investment,Shares Added,Shares Total,Target Sell Price
$8.00,($0.00),,,$18.18
$8.00,$860.85,,,$9.19


Current Share Price,Addl Investment,Shares Added,Shares Total,Target Sell Price
$9.09,$0.00,0,110,$10.00
$8.00,$96.00,12,122,$9.02
$8.00,$200.00,25,135,$8.15
$8.00,$496.00,62,172,$6.40


Current Share Price,Addl Investment,Shares Added,Shares Total,Target Sell Price
$8.00,($396.00),,,$18.18
$8.00,$77.47,,,$9.19


In [ ]:
##### CALCULATE TAXES ON CAPITAL GAINS ##############

import locale
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

locale.setlocale(locale.LC_ALL, '')

def format_currency(amount):
    return locale.currency(amount, grouping=True)


def calc_magi(annual_income, capital_gains):
    return annual_income + capital_gains


# NEW RATES 2025
# Short-Term Rates (Single) - Ordinary Income Tax Brackets 2025
SHORT_TERM_RATES_SINGLE = [
    (0, 11600, 0.10),         # 10% up to $11,600
    (11601, 47150, 0.12),     # 12% from $11,601 to $47,150
    (47151, 100525, 0.22),    # 22% from $47,151 to $100,525
    (100526, 191950, 0.24),   # 24% from $100,526 to $191,950
    (191951, 243700, 0.32),   # 32% from $191,951 to $243,700
    (243701, 609350, 0.35),   # 35% from $243,701 to $609,350
    (609351, float('inf'), 0.37) # 37% over $609,350
]

# Long-Term Rates (Single) - Capital Gains 2025
LONG_TERM_RATES_SINGLE = [
    (0, 48350, 0.00),         # 0% up to $48,350
    (48351, 533400, 0.15),    # 15% from $48,351 to $533,400
    (533401, float('inf'), 0.20) # 20% over $533,400
]

# Short-Term Rates (Married Filing Jointly) - Ordinary Income Tax Brackets 2025
SHORT_TERM_RATES_MARRIED = [
    (0, 23200, 0.10),         # 10% up to $23,200
    (23201, 94300, 0.12),     # 12% from $23,201 to $94,300
    (94301, 201050, 0.22),    # 22% from $94,301 to $201,050
    (201051, 383900, 0.24),   # 24% from $201,051 to $383,900
    (383901, 487450, 0.32),   # 32% from $383,901 to $487,450
    (487451, 731200, 0.35),   # 35% from $487,451 to $731,200
    (731201, float('inf'), 0.37) # 37% over $731,200
]

# Long-Term Rates (Married Filing Jointly) - Capital Gains 2025
LONG_TERM_RATES_MARRIED = [
    (0, 96700, 0.00),         # 0% up to $96,700
    (96701, 600050, 0.15),    # 15% from $96,701 to $600,050
    (600051, float('inf'), 0.20) # 20% over $600,050
]


def calc_NIIT(capital_gains: float, magi: float, marital: Marital = Marital.SINGLE) -> float:
    """
    Calculate the Net Investment Income Tax (NIIT).
    
    The NIIT imposes a 3.8% surtax on the lesser of:
    - Your net investment income (capital gains), or
    - The amount by which your AGI exceeds the applicable threshold.
    
    Returns the amount of NIIT owed.
    """
    NIIT_RATE = 0.038
    SINGLE_MAGI_THRESHOLD = 200000
    MARRIED_JOINTLY_MAGI_THRESHOLD = 250000

    if marital == Marital.MARRIED:
        if magi <= MARRIED_JOINTLY_MAGI_THRESHOLD:
            return 0
        exceeded_income = magi - MARRIED_JOINTLY_MAGI_THRESHOLD
    else: # assume Single marital status
        if magi <= SINGLE_MAGI_THRESHOLD:
            return 0
        exceeded_income = magi - SINGLE_MAGI_THRESHOLD

    taxable_amt = min(max(0,capital_gains), exceeded_income) # prevent negative capital gains

    return taxable_amt * NIIT_RATE


def find_rate(magi: float, tax_rates) -> float|None:
    for lower_bound, upper_bound, rate in tax_rates:
        if lower_bound <= magi <= upper_bound:
            return rate
    return None 

def get_tax_rate(capital_gains: float, marital: Marital, holding_period: HoldingPeriod) -> float:
    rates = None

    if holding_period == HoldingPeriod.SHORTTERM:
        if marital == Marital.SINGLE:
            rates = SHORT_TERM_RATES_SINGLE
        elif marital == Marital.MARRIED:
            rates = SHORT_TERM_RATES_MARRIED

    elif holding_period == HoldingPeriod.LONGTERM:
        if marital == Marital.SINGLE:
            rates = LONG_TERM_RATES_SINGLE
        elif marital == Marital.MARRIED:
            rates = LONG_TERM_RATES_MARRIED

    if rates is None:
        raise ValueError("No tax rates found given the passed in information")

    # Calculate the effective tax rate based on progressive brackets for capital gains
    total_tax = 0.0
    previous_limit = 0.0

    for lower_bound, upper_bound, rate in rates:
        if capital_gains > upper_bound:
            # Tax the full range of this bracket
            total_tax += (upper_bound - previous_limit) * rate
            previous_limit = upper_bound
        else:
            # Tax only the portion of capital gains that falls within this bracket
            if capital_gains > lower_bound:
                total_tax += (capital_gains - previous_limit) * rate
            break  # No need to check further brackets

    effective_tax_rate = total_tax / capital_gains if capital_gains > 0 else 0.0
    return effective_tax_rate

def calculate_tax_on_capital_gains(marital: Marital, annual_income: float, capital_gains: float, holding_period: HoldingPeriod):
    effective_tax_rate = get_tax_rate(capital_gains, marital, holding_period)

    magi = calc_magi(annual_income, capital_gains) # modified adjusted gross income
    NIIT = calc_NIIT(capital_gains, magi, marital)

    print(f'Effective Tax Rate for {format_currency(capital_gains)} in {holding_period} Capital Gains: {round(effective_tax_rate*100)}%.')
    print(f'Capital Gains Tax Amt: {format_currency(capital_gains * effective_tax_rate)}')
    print(f'NIIT amount: {format_currency(NIIT)}')

    return capital_gains * effective_tax_rate + NIIT

capital_gains = 10000
annual_income = 50000

tax = calculate_tax_on_capital_gains(
    marital=Marital.SINGLE,
    annual_income=annual_income,
    capital_gains=capital_gains,
    holding_period=HoldingPeriod.SHORTTERM
)

statetax = capital_gains * STATE_TAX_RATE

print(f'Tax Owed -- Total: {format_currency(tax + statetax)}, Federal: {format_currency(tax)}, State: {format_currency(statetax)}')
print(f'Investment worth before tax: {format_currency(TOTAL_INITIAL_INVESTMENT + capital_gains)}')
print(f'Investment worth after tax: {format_currency(TOTAL_INITIAL_INVESTMENT + capital_gains - tax - statetax)}')

# should be 64k, or not? check https://smartasset.com/investing/capital-gains-tax-calculator#4I24Z617rf - that shows a different val??

Effective Tax Rate for $10,000.00 in Short Term Capital Gains: 10%.
Capital Gains Tax Amt: $1,000.00
NIIT amount: $0.00
Tax Owed -- Total: $1,440.00, Federal: $1,000.00, State: $440.00
Investment worth before tax: $11,000.00
Investment worth after tax: $9,560.00


In [ ]:
# Interactive function to visualize tax owed based on parameters
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  
import numpy as np

def format_func(value, tick_number):
    if value >= 1000000:
        return f'{value / 1000000:.0f}M'  # Convert to millions
    elif value >= 1000:
        return f'{value / 1000:.0f}k'  # Convert to thousands
    else:
        return f'{int(value)}'  # Return the value as is


def visualize_taxes(annual_income, final_market_val, initial_cost, holding_period, marital_status, state_tax_rate):
    capital_gains = final_market_val - initial_cost

    holdingperiod = HoldingPeriod.SHORTTERM if holding_period == 0 else HoldingPeriod.LONGTERM
    maritalstatus = Marital.SINGLE if marital_status == 0 else Marital.MARRIED
    
    tax_owed = calculate_tax_on_capital_gains(marital=maritalstatus, annual_income=annual_income, capital_gains=capital_gains, holding_period=holdingperiod)
    statetax = capital_gains * (state_tax_rate / 100)

    after_tax_value = final_market_val - tax_owed - statetax
    
    plt.figure(figsize=(10,5))
    # plt.bar(['Total Tax Owed'], [tax_owed], color='blue')

    plt.plot(final_market_val, after_tax_value, 'o', markersize=15, label=f'After Tax Value = {format_currency(after_tax_value)}')

    plt.xlabel('Final Market Value ($)')
    plt.ylabel('After Tax Value ($)')

    MAX_X = final_market_val + 1000 if final_market_val < 100_000 else 1_000_000
    MAX_Y = after_tax_value + 1000 if after_tax_value < 100_000 else 1_000_000
    plt.xticks(np.arange(0, MAX_X+1, MAX_X / 10))
    plt.xlim(0, MAX_X)
    plt.ylim(0, MAX_Y)
    
    # Set the x-ticks to show 100k 200k etc.
    if MAX_X > 100_000:
        plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_func))
    plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_func))

    plt.legend()
    plt.show()


desired_market_val = 2000
annual_income = 50000

# Create sliders for interactive parameters
annual_income_slider = widgets.IntSlider(value=annual_income, min=0, max=200000,
                                         step=1000, description='Annual Income:',
                                         style={'description_width': 'initial'},
                                         layout=widgets.Layout(width='500px'))

final_market_val_slider = widgets.IntSlider(value=desired_market_val, min=0,
                                             max=desired_market_val + 1000, step=desired_market_val / 100,
                                             description='Final Market Value:',
                                             style={'description_width': 'initial'},
                                             layout=widgets.Layout(width='500px'))

initial_cost_slider = widgets.IntSlider(value=TOTAL_INITIAL_INVESTMENT, min=0,
                                             max=1000000, step=1000,
                                             description='Initial Cost:',
                                             style={'description_width': 'initial'},
                                             layout=widgets.Layout(width='500px'))

holding_period_slider = widgets.IntSlider(value=0, min=0,
                                             max=1, step=1,
                                             description='Holding period (0 - short, 1 - long):',
                                             style={'description_width': 'initial'},
                                             layout=widgets.Layout(width='500px'))

marital_slider = widgets.IntSlider(value=0, min=0,
                                             max=1, step=1,
                                             description='Marital Status (0 - Single, 1 - Married):',
                                             style={'description_width': 'initial'},
                                             layout=widgets.Layout(width='500px'))

state_tax_rate_slider = widgets.FloatSlider(value=STATE_TAX_RATE*100, min=0, max=10, 
                    step=0.01, readout_format='.2f', description='% State Tax')

# Use interactive output to visualize changes
widgets.interactive(visualize_taxes,
                    annual_income=annual_income_slider,
                    final_market_val=final_market_val_slider,
                    initial_cost=initial_cost_slider,
                    holding_period=holding_period_slider,
                    marital_status=marital_slider,
                    state_tax_rate=state_tax_rate_slider)

interactive(children=(IntSlider(value=50000, description='Annual Income:', layout=Layout(width='500px'), max=2…

In [12]:
#### PLOT TARGET PRICE #########

import locale
import matplotlib.ticker as ticker  
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

locale.setlocale(locale.LC_ALL, '')

def format_currency(amount):
    return locale.currency(amount, grouping=True)

def format_func(value, tick_number):
    if value >= 1000000:
        return f'{value / 1000000:.0f}M'  # Convert to millions
    elif value >= 1000:
        return f'{value / 1000:.0f}k'  # Convert to thousands
    else:
        return f'{int(value)}'  # Return the value as is


def investment_required_for_target_price(desired_market_val, future_price, purchase_price, current_shares_owned, current_cost_basis_per_share):

    addl_investment = required_investment_amt_for(
        desired_market_val=desired_market_val, 
        target_price=future_price, 
        purchase_price=purchase_price,
        current_shares_owned=current_shares_owned
    )

    print(f'Current Market Value: {format_currency(current_shares_owned * current_cost_basis_per_share)}')
    
    plt.figure(figsize=(10,5))

    plt.plot(future_price, addl_investment, 'o', markersize=15, label=f'Addl Investment Needed: {format_currency(addl_investment)}')

    plt.xlabel('Target Price ($)')
    plt.ylabel('Addl Investment ($)')

    MIN_X = 0
    MAX_X = future_price + 100

    MIN_Y = addl_investment - 10000 if addl_investment < 0 else 0
    MAX_Y = addl_investment + 1000 if addl_investment < 100000 else addl_investment + 100000

    plt.xlim(MIN_X, MAX_X)
    plt.ylim(MIN_Y, MAX_Y)

    # plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_func))
    plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_func))

    plt.legend()
    plt.title(r'To return the desired market value of \${:,.2f} if the Future Share Price reaches \${:,.2f}'.format(desired_market_val, future_price))
    plt.show()

likely_future_price = 10.00

# Create sliders for interactive parameters
future_share_price_slider = widgets.FloatSlider(value=likely_future_price, min=0, max=100, 
                    step=0.01, readout_format='.2f', description='If Share Price Gets To:',
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='500px'))

purchase_price_slider = widgets.FloatSlider(value=CURRENT_SHARE_PRICE, min=0, max=100, 
                    step=0.01, readout_format='.2f', description='Given Shares Purchased Now At:',
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='500px'))

current_market_value = TOTAL_SHARES_OWNED * CURRENT_COST_BASIS_PER_SHARE
desired_market_value_slider = widgets.IntSlider(value=FINAL_MARKET_VAL, min=current_market_value,
                                             max=1_000_000, step=1000,
                                             description='I Want to End Up With ($):',
                                             style={'description_width': 'initial'},
                                             layout=widgets.Layout(width='500px'))

shares_owned_slider = widgets.IntSlider(value=TOTAL_SHARES_OWNED, min=0,
                                             max=10_000, step=1,
                                             description='While I Currently Own This Num Shares:',
                                             style={'description_width': 'initial'},
                                             layout=widgets.Layout(width='500px'))

# TODO: This is hidden now, but could add to function to change other values when this changes, currently not used accept to print current market val
current_cost_basis_per_share_slider = widgets.FloatSlider(
    value=CURRENT_COST_BASIS_PER_SHARE, 
    min=0, 
    max=CURRENT_COST_BASIS_PER_SHARE + 50 if CURRENT_COST_BASIS_PER_SHARE else 100,
    step=0.01, 
    readout_format='.2f', 
    description='With a Current Cost Basis (Share) of:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px'),
    readout=True,  # Set to True to show the value
    continuous_update=False,
    disabled=True  # This disables the slider
)

# Use interactive output to visualize changes
interactive_plot = widgets.interactive(investment_required_for_target_price,
    desired_market_val=desired_market_value_slider,
    future_price=future_share_price_slider,
    purchase_price=purchase_price_slider,
    current_shares_owned=shares_owned_slider,
    current_cost_basis_per_share=current_cost_basis_per_share_slider
)

# Display the interactive plot
display(interactive_plot)

interactive(children=(IntSlider(value=2000, description='I Want to End Up With ($):', layout=Layout(width='500…